# K-means clustering with Heratrate, Steps, and SleepMin factors

### Steps to set up graphviz for decision tree classification
1. Download and Install graphviz for windows at https://graphviz.gitlab.io/_pages/Download/Download_windows.html
2. Install the libraries
    - conda install -c anaconda graphviz
    - conda install -c anaconda pydotplus
3. Add to path in system Variables
    - type "Edit the system environment variables"
    - In Advanced tab go to Environment Variables
    - In System variables edit the Path and add _C:\Program Files (x86)\Graphviz2.38\bin_ or location of the bin where you previously installed grahviz

In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
#For Descision Tree Visualization
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz 
import pydotplus

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-6f5a3c7e9e0a>", line 5, in <module>
    from sklearn.datasets.samples_generator import make_blobs
  File "//anaconda3/lib/python3.7/site-packages/sklearn/datasets/__init__.py", line 22, in <module>
    from .twenty_newsgroups import fetch_20newsgroups
  File "//anaconda3/lib/python3.7/site-packages/sklearn/datasets/twenty_newsgroups.py", line 44, in <module>
    from ..feature_extraction.text import CountVectorizer
  File "//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/__init__.py", line 10, in <module>
    from . import text
  File "//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py", line 28, in <module>
    from ..preprocessing import normalize
  File "//anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/__init__.p

TypeError: __call__() got an unexpected keyword argument 'warn_on_impl'

In [57]:
df_j3 = pd.read_csv("finalData.csv",index_col=0)
df_j3["Activities"]=df_j3["VeryActiveMinutes"] +df_j3["FairlyActiveMinutes"]+df_j3["LightlyActiveMinutes"]
df_j3.drop(['VeryActiveMinutes', 'FairlyActiveMinutes','LightlyActiveMinutes','Calories',
            'Activities' , 'SedentaryMinutes' ], axis=1, inplace = True)
df_j3

,Heartrate,TotalSteps,sleepMin
0,64.863768,8328,528
1,61.058824,8328,528
2,59.720930,8328,528
3,58.935103,8328,528
4,58.590116,8328,528
...,...,...,...
1251,81.772257,20,492
1252,67.000000,20,492
1253,62.180556,20,492
1254,66.549072,20,492


In [58]:
# transforme data into standard scaler
from sklearn.preprocessing import StandardScaler
X = df_j3.values
X = np.nan_to_num(X)
Clus_dataSet = StandardScaler().fit_transform(X)
Clus_dataSet

array([[-0.66172326, -0.23068574,  0.7685682 ],
       [-0.94626452, -0.23068574,  0.7685682 ],
       [-1.04631482, -0.23068574,  0.7685682 ],
       ...,
       [-0.86237919, -2.11812539,  0.47050676],
       [-0.53569295, -2.11812539,  0.47050676],
       [-0.26184259, -2.11812539,  0.47050676]])

In [59]:
#elbow method
Sum_of_squared_distances = []
K = range(1,12)
for k in K:
    km = KMeans(init = "k-means++", n_clusters = k)
    km = km.fit(Clus_dataSet)
    Sum_of_squared_distances.append(km.inertia_)

In [1]:
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

NameError: name 'plt' is not defined

elbow method showed that the optimal k would be 4

In [78]:
clusterNum = 4
k_means = KMeans(init = "k-means++", n_clusters = clusterNum)
k_means.fit(Clus_dataSet)
labels = k_means.labels_
print(labels)

[0 0 0 ... 2 2 2]


In [83]:
df_j3["Label"] = labels
df_j3["Label"] = pd.Categorical(df_j3["Label"], df_j3["Label"].unique())
df_j3["Label"] = df_j3["Label"].cat.rename_categories(['First', 'Second', 'Third', 'Fourth'])
df_j3.head(5)

,Heartrate,TotalSteps,sleepMin,Label
0,64.863768,8328,528,First
1,61.058824,8328,528,First
2,59.720930,8328,528,First
3,58.935103,8328,528,First
4,58.590116,8328,528,First


In [84]:
# used to store the cluster data
#df_j3.to_csv('clustered_data.csv')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 
fig = plt.figure(1, figsize=(8, 6))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=38, azim=135)

plt.cla()

ax.set_xlabel('Heart Rate')
ax.set_ylabel('Total Step')
ax.set_zlabel('Sleep Min')

ax.scatter(X[:, 0], X[:, 1], X[:, 2], c= labels.astype(np.float),alpha=1)

In [88]:
first_cluster = df_j3.loc[df_j3['Label']=="First"]

second_cluster = df_j3.loc[df_j3['Label']=="Second"]

third_cluster = df_j3.loc[df_j3['Label']=="Third"]

fourth_cluster = df_j3.loc[df_j3['Label']=="Fourth"]


In [89]:
first_cluster.describe()

,Heartrate,TotalSteps,sleepMin
count,578.000000,578.000000,578.000000
mean,66.579713,11433.103806,445.536332
std,7.200612,2379.047537,61.557116
min,43.750799,7401.000000,256.000000
25%,62.117952,10001.000000,405.000000
50%,67.116053,11080.000000,445.000000
75%,72.202944,12409.000000,480.000000
max,79.189266,19658.000000,562.000000


#### From the first cluster we can see that the heartrate is normal, the steps are very good, and the sleeping minures are normal. Therefore, we can say that the chance that the user might have anxiety is [Low].

In [90]:
second_cluster.describe()

,Heartrate,TotalSteps,sleepMin
count,231.000000,231.000000,231.000000
mean,90.991107,11717.610390,466.632035
std,9.834354,3276.150985,57.017607
min,78.032828,2862.000000,352.000000
25%,83.655489,10114.000000,422.000000
50%,88.814672,11761.000000,458.000000
75%,95.782721,12480.000000,515.000000
max,127.679529,19658.000000,617.000000


#### From the second cluster we can see that 75% of the heartrate is high, the sleeping minutes are very low, but the steps are very good. Therefore, we can say that the chance that the user might have anxiety is [High].

In [91]:
third_cluster.describe()

,Heartrate,TotalSteps,sleepMin
count,348.000000,348.000000,348.000000
mean,70.585305,3629.497126,491.428161
std,10.958806,2389.406322,85.865272
min,46.793750,8.000000,363.000000
25%,62.899650,1988.000000,450.000000
50%,70.074113,3436.000000,468.000000
75%,77.774612,5893.000000,516.000000
max,101.370667,8328.000000,750.000000


#### From the third cluster we can see that the heartrate is normal, the steps are very weak, and the sleeping minutes are normal. Therefore, we can say that the chance that the user might have anxiety is [Medium].

In [92]:
fourth_cluster.describe()

,Heartrate,TotalSteps,sleepMin
count,99.000000,99.000000,99.000000
mean,86.031864,11688.525253,103.505051
std,11.934618,3009.225035,74.748609
min,62.240356,7476.000000,63.000000
25%,77.779223,9910.000000,65.000000
50%,84.526462,10976.000000,69.000000
75%,93.424594,14100.000000,72.000000
max,128.607407,16806.000000,256.000000


#### From the fourth cluster we can see that 75% of the heartrate is high, the steps are very good, and the sleeping minutes are normal. Therefore, we can say that the chance that the user might have anxiety is [Low]. Note: The user might be active, which explains the high heartrate.

# Step By Step Guide to Descision Tree Classifier

Assigned X to feature varaiables

In [95]:
df_desc=pd.read_csv('clustered_data.csv',index_col=0)
df_desc.head()

,Heartrate,TotalSteps,sleepMin,Label
0,64.863768,8328,528,First
1,61.058824,8328,528,First
2,59.720930,8328,528,First
3,58.935103,8328,528,First
4,58.590116,8328,528,First


In [96]:
X = df_desc[['Heartrate', 'TotalSteps' , 'sleepMin']].values
X[0:5]

array([[  64.86376812, 8328.        ,  528.        ],
       [  61.05882353, 8328.        ,  528.        ],
       [  59.72093023, 8328.        ,  528.        ],
       [  58.93510324, 8328.        ,  528.        ],
       [  58.59011628, 8328.        ,  528.        ]])

Assigned y to target varaiable (Label)

In [97]:
y = df["Label"]
y[0:5]

0    First
1    First
2    First
3    First
4    First
Name: Label, dtype: object

### Splitting of Data
Split the data into testing and training set and took 30% of data as testing and 70% for training the model

In [98]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3)

In [99]:
print(np.shape(X_trainset))
print(np.shape(y_trainset))

(879, 3)
(879,)


In [100]:
print(np.shape(X_testset))
print(np.shape(y_testset))

(377, 3)
(377,)


Define the model, we can adjust the max_depth of node of decission tree, default value is set to None, it means our decission tree classifier would generate only pure node

In [101]:
descTree = DecisionTreeClassifier(criterion="entropy",max_depth=None)
descTree

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

fit the training model in the our descTree model and predict with test set features

In [110]:
descTree.fit(X_trainset,y_trainset)
predTree = descTree.predict(X_testset)

#### Accuracy of Classifier

In [111]:
print (predTree [0:5])
print (y_testset [0:5])

['First' 'First' 'First' 'Third' 'First']
641     First
76      First
851     First
335    Second
899     First
Name: Label, dtype: object


In [113]:
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

DecisionTrees's Accuracy:  0.9840848806366048


### Decision Tree Visualization

In [ ]:
dot_data = StringIO()
export_graphviz(descTree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,
                feature_names=df_desc.columns[0:3],  
                class_names=df_desc.Label.unique().tolist())
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

We can adjust the depth of decision tree(number of branches) by setting the _max_depth_ value in our model
- $X_{0}$ is first feature Heart Rate
- $X_{1}$ is second feature Total Steps
- $X_{2}$ is thrid feature sleepMin

## Classification for new data set

In [143]:
# Step 1
df=pd.read_csv('clustered_data.csv',index_col=0)    # Load label data
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3)  #Splitting of Data into train and test set
descTree = DecisionTreeClassifier(criterion="entropy",max_depth=None) # Define Model
descTree.fit(X_trainset,y_trainset) # train the model using training set
predTree = descTree.predict(X_testset)  # test the model using testing set
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

DecisionTrees's Accuracy:  0.9840848806366048


Now our model is trained using 3 parameters _Heartrate_ ,  _TotalSteps_ and  _sleepMin_. Make sure to load the data with same parameters

In [ ]:
# Visualization
dot_data = StringIO()
export_graphviz(descTree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,
                feature_names=df.columns[0:3],  
                class_names=df.Label.unique().tolist())
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [140]:
# Labeling of new data set
new_df=pd.read_csv('****.csv',index_col=0) # load the csv file
X = new_df[['Heartrate', 'TotalSteps' , 'sleepMin']].values
new_df["Label"] = descTree.predict(X)
new_df.head(10)